# CMOR example with prototype CMIP7 tables/CVs

This code is a demonstration of the use of CMOR with prototype CMIP7 CVs and tables. This will require a conda environment in which CMOR 3.10.0 has been installed

Please note that file and directory names required for CMIP7 have not been confirmed yet, this is just a limited demonstration.

In [1]:
%time
import cmor
import numpy 
import json

CPU times: user 2 μs, sys: 1 μs, total: 3 μs
Wall time: 5.48 μs


The following is adapted from the code at https://github.com/PCMDI/cmor/blob/main/Test/test_cmor_CMIP7.py

Construct the input json file for CMOR


In [2]:
%time
DATASET_INFO = {
    "_AXIS_ENTRY_FILE": "development-tables/CMIP7_coordinate.json",
    "_FORMULA_VAR_FILE": "development-tables/CMIP7_formula_terms.json",
    "_cmip7_option": 1,
    "_controlled_vocabulary_file": "development-tables/CMIP7_CV.json",
    "activity_id": "CMIP",
    "branch_method": "standard",
    "branch_time_in_child": 30.0,
    "branch_time_in_parent": 10800.0,
    "calendar": "360_day",
    "cv_version": "7.0.0.0",
    "experiment": "1 percent per year increase in CO2",
    "experiment_id": "1pctCO2",
    "forcing_index": "3",
    "grid": "N96",
    "grid_label": "gn",
    "initialization_index": "1",
    "institution_id": "PCMDI",
    "license": "CC BY 4.0",
    "nominal_resolution": "250 km",
    "outpath": ".",
    "parent_mip_era": "CMIP7",
    "parent_time_units": "days since 1850-01-01",
    "parent_activity_id": "CMIP",
    "parent_source_id": "PCMDI-test-1-0",
    "parent_experiment_id": "piControl",
    "parent_variant_label": "r1i1p1f3",
    "physics_index": "1",
    "realization_index": "9",
    "source_id": "PCMDI-test-1-0",
    "source_type": "AOGCM CHEM BGC",
    "tracking_prefix": "hdl:21.14100",
    "host_collection": "CMIP7",
    "frequency": "day",
    "region": "glb",
    "archive_id": "WCRP",
    "mip_era": "CMIP7",
    "output_path_template": "<mip_era><activity_id><source_id><experiment_id><member_id><realm><frequency><variable_id><branding_suffix><grid_label><version>",
    "output_file_template": "<variable_id><branding_suffix><frequency><region><grid_label><source_id><experiment_id><variant_label>",
}

with open('input.json', 'w') as fh:
    json.dump(DATASET_INFO, fh, indent=2)

CPU times: user 2 μs, sys: 1 μs, total: 3 μs
Wall time: 5.01 μs


In [3]:
%time
cmor.setup(inpath="development-tables", netcdf_file_action=cmor.CMOR_REPLACE)

cmor.dataset_json('input.json')

tos = numpy.array([27, 27, 27, 27,
                    27, 27, 27, 27,
                    27, 27, 27, 27,
                    27, 27, 27, 27,
                    27, 27, 27, 27,
                    27, 27, 27, 27
                    ])
tos.shape = (2, 3, 4)
lat = numpy.array([10, 20, 30])
lat_bnds = numpy.array([5, 15, 25, 35])
lon = numpy.array([0, 90, 180, 270])
lon_bnds = numpy.array([-45, 45,
                        135,
                        225,
                        315
                        ])
time = numpy.array([15.5, 16.5])
time_bnds = numpy.array([15, 16, 17])

CPU times: user 1 μs, sys: 0 ns, total: 1 μs
Wall time: 4.53 μs


In [ ]:
%time
cmor.load_table("CMIP7_ocean.json")

In [ ]:
%time
cmorlat = cmor.axis("latitude",
                    coord_vals=lat,
                    cell_bounds=lat_bnds,
                    units="degrees_north")
cmorlon = cmor.axis("longitude",
                    coord_vals=lon,
                    cell_bounds=lon_bnds,
                    units="degrees_east")
cmortime = cmor.axis("time",
                    coord_vals=time,
                    cell_bounds=time_bnds,
                    units="days since 2018")
axes = [cmortime, cmorlat, cmorlon]

In [ ]:
cmortos = cmor.variable("tos_tavg-u-hxy-sea", "degC", axes)

In [ ]:
cmor.write(cmortos, tos)
filename = cmor.close(cmortos, file_name=True)

Locate generated file

In [ ]:
!find CMIP7 -type f -name *.nc

Dump header of generated file

In [ ]:
!ncdump -h CMIP7/CMIP/PCMDI-test-1-0/1pctCO2/r9i1p1f3/ocean/day/tos/tavg-u-hxy-sea/gn/*/tos_tavg-u-hxy-sea_day_glb_gn_PCMDI-test-1-0_1pctCO2_r9i1p1f3_20180116-20180117.nc

In [ ]:
# clean up
! rm -rf CMIP7
! rm -rf input.json